In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
#import missingno as msno
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
import plotly.express as px

df=pd.read_csv(r"/content/drive/MyDrive/OSMI/mental_health_dataset.csv")
df17=pd.read_excel("/content/drive/MyDrive/OSMI/Mental-Health-in-Tech/Data/OSMI 2017.xlsx")
df18=pd.read_excel("/content/drive/MyDrive/OSMI/Mental-Health-in-Tech/Data/OSMI 2018.xlsx")
df19=pd.read_excel("/content/drive/MyDrive/OSMI/Mental-Health-in-Tech/Data/OSMI 2019 .xlsx")
df20=pd.read_excel("/content/drive/MyDrive/OSMI/Mental-Health-in-Tech/Data/OSMI 2020.xlsx")
df21=pd.read_excel("/content/drive/MyDrive/OSMI/Mental-Health-in-Tech/Data/OSMI 2021.xlsx")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

Indepedent variables that can be used: Age, gender, family history and benefits offered by employer


In [4]:
df=pd.read_csv("/content/drive/MyDrive/OSMI/mental_health_dataset.csv")
for col in df.columns:
    print(col)

Are you self-employed?
How many employees does your company or organization have?
Is your employer primarily a tech company/organization?
Is your primary role within your company related to tech/IT?
Does your employer provide mental health benefits as part of healthcare coverage?
Do you know the options for mental health care available under your employer-provided coverage?
Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?
Does your employer offer resources to learn more about mental health concerns and options for seeking help?
Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?
If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:
Do you think that discussing a mental health disorder with your employer would have negative consequences?
Do you think that disc

In [5]:
renamed_columns=["Self_employed","Total_employees","Tech_organization","IT_Tech","Mental_health_coverage","Awareness_of_options_under_coverage","Employers_discussion_on_mental_health","Resources_and_options_for_help","Anomity_of_employees_using_mental_health_resources","Mental_issue_related_leave","Discussing_mental_health_neg_consequences","Discussing_physical_health_neg_consequences","Comfort_discussing_with_coworkers","Comfort_discussing_with_supervisor","Employer_seriousness_on_mental_health","Negative_consequences_of_coworkers_with_mental_health_disorder","Medical_coverage","Aware_of_local_online_resources","Reveal_condition_to_clients_business_contacts","Consequence_if_revealed_health_issue_to_client_or_business_contact",'Diagnosed_but_not_revealed_with_coworkers_employees','negimpact_reveal_coworker','prod_affect_mental_issue','percent_worktime_affected','have_prev_empl','prev_empl_mental_benefits','aware_benefits_prev_empl','formal_discuss_mental_health_premp','premp_provide_resources_learn_seekhelp','anonymity_protect_premp_choose_resources','neg_eff_discuss_mental_health_premp','neg_eff_discuss_phy_health_premp','willing_discuss_issue_prev_coworkers','willing_discuss_issue_dir_supervisors','premp_took_menhealth_seriously_as_phyhealth','neg_comments_abt_coworkers_with_menissues_prev_workplace','bring_phyhealth_issue_with_potential_empl_interview','phy_y_ynot','bring_menhealth_issue_with_potential_empl_interview','men_y_ynot','hurt_carrer_if_identified_as_person_with_mental_issue','more_neg_view_from_coworker_if_know_abt_men_issue','share_w_friends_fam','workplace_response','other_exp_in_workplace','fam_hist','past_mh','curr_mh','diagnosed','maybe_diagnosed','diagnosed_prof','diagnosed_prof_mh','treatment_mh_prof','work_interference_treatment','work_interference_no_treatment','age','gender','country_live','us_state_live','country_work','us_state_work','work_pos','remote']
df.columns=renamed_columns

In [6]:

to_drop_columns=[i for i in df.columns if df[i].isna().sum()>df.shape[0]/2]
print(to_drop_columns)
df.drop(to_drop_columns,axis=1,inplace=True)

['IT_Tech', 'Medical_coverage', 'Aware_of_local_online_resources', 'Reveal_condition_to_clients_business_contacts', 'Consequence_if_revealed_health_issue_to_client_or_business_contact', 'Diagnosed_but_not_revealed_with_coworkers_employees', 'negimpact_reveal_coworker', 'prod_affect_mental_issue', 'percent_worktime_affected', 'other_exp_in_workplace', 'diagnosed', 'maybe_diagnosed', 'diagnosed_prof_mh']


In [7]:
#gender has 70 different classes. Encoding them to fewer classes for ease of calculation:
#male:1, female:2, anything else:3
df['gender'].replace(to_replace = ['Male', 'male', 'Male ', 'M', 'm',
       'man', 'Cis male', 'Male.', 'male 9:1 female, roughly', 'Male (cis)', 'Man', 'Sex is male',
       'cis male', 'Malr', 'Dude', "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
       'mail', 'M|', 'Male/genderqueer', 'male ',
       'Cis Male', 'Male (trans, FtM)',
       'cisdude', 'cis man', 'MALE'], value = 1, inplace = True)

df['gender'].replace(to_replace = ['Female', 'female', 'I identify as female.', 'female ',
       'Female assigned at birth ', 'F', 'Woman', 'fm', 'f', 'Cis female ', 'Transitioned, M2F',
       'Genderfluid (born female)', 'Female or Multi-Gender Femme', 'Female ', 'woman', 'female/woman','Cisgender Female', 'fem', 'Female (props for making this a freeform field, though)',
       ' Female', 'Cis-woman', 'female-bodied; no feelings about gender',
       'AFAB'], value = 2, inplace = True)

df['gender'].replace(to_replace = ['Bigender', 'non-binary', 'Other/Transfeminine',
       'Androgynous', 'Other', 'nb masculine',
       'none of your business', 'genderqueer', 'Human', 'Genderfluid',
       'Enby', 'genderqueer woman', 'mtf', 'Queer', 'Agender', 'Fluid',
       'Nonbinary', 'human', 'Unicorn', 'Genderqueer',
       'Genderflux demi-girl', 'Transgender woman','NaN'], value = 3, inplace = True)
#Change all non-responses to 3
df['gender'].replace(to_replace=[np.NaN], value=3,inplace=True)

In [8]:
#columns 'phy_y_ynot' and 'men_y_ynot' mainly contain comments which aren't of much use for our problem definition
# hence, we drop the columns 'phy_y_ynot' and 'men_y_ynot'
df.drop(labels ='phy_y_ynot', axis = 1, inplace = True)
df.drop(labels ='men_y_ynot', axis = 1, inplace = True)

# dealing with missing values in 'prev_empl_mental_benefits'
# from previous operation, we see that, whenever the value for have_prev_empl = 0, then the value for 'prev_empl_mental_benefits' = Nan
# hence replacing the nans with "Not Applicable"
df['prev_empl_mental_benefits'].fillna('Not Applicable',inplace= True)
df['prev_empl_mental_benefits'].unique()

# the same observation was found for the missing values in columns from formal_discuss_mental_health_premp' to'neg_comments_abt_coworkers_with_menissues_prev_workplace'
# replacing missing values
# whenever the value for have_prev_empl = 0, then the value in the other col= Nan
# hence replacing the nans with "Not Applicable"
df['aware_benefits_prev_empl'].fillna('Not Applicable',inplace= True)
df['formal_discuss_mental_health_premp'].fillna('Not Applicable',inplace= True)
df['premp_provide_resources_learn_seekhelp'].fillna('Not Applicable',inplace= True)
df['anonymity_protect_premp_choose_resources'].fillna('Not Applicable',inplace= True)
df['neg_eff_discuss_mental_health_premp'].fillna('Not Applicable',inplace= True)
df['neg_eff_discuss_phy_health_premp'].fillna('Not Applicable',inplace= True)
df['willing_discuss_issue_dir_supervisors'].fillna('Not Applicable',inplace= True)
df['willing_discuss_issue_prev_coworkers'].fillna('Not Applicable',inplace= True)
df['premp_took_menhealth_seriously_as_phyhealth'].fillna('Not Applicable',inplace= True)
df['neg_comments_abt_coworkers_with_menissues_prev_workplace'].fillna('Not Applicable',inplace= True)

#Acronyms of countries with long names (United states of america)
df['country_live'].replace(to_replace = ['United States of America'], value = 'USA', inplace = True)
df['country_live'].replace(to_replace = ['United Kingdom'], value = 'UK', inplace = True)
df['country_work'].replace(to_replace = ['United States of America'], value = 'USA', inplace = True)
df['country_work'].replace(to_replace = ['United Kingdom'], value = 'UK', inplace = True)

#Removing age outliers (323,3,etc). Replace such ages with the mean of all ages
mean_age = df[(df['age'] >= 18) | (df['age'] <= 80)]['age'].mean()
df['age'].replace(to_replace = df[(df['age'] < 18) | (df['age'] > 75)]['age'].tolist(),
                          value = mean_age, inplace = True)
#df=df.dropna()

In [9]:
df.isnull().sum().sum()

32490

In [10]:

#Remaining columns with null values
for i in df.columns:
    if(df[i].isnull().values.any()):
        print(i)

Total_employees
Tech_organization
Mental_health_coverage
Awareness_of_options_under_coverage
Employers_discussion_on_mental_health
Resources_and_options_for_help
Anomity_of_employees_using_mental_health_resources
Mental_issue_related_leave
Discussing_mental_health_neg_consequences
Discussing_physical_health_neg_consequences
Comfort_discussing_with_coworkers
Comfort_discussing_with_supervisor
Employer_seriousness_on_mental_health
Negative_consequences_of_coworkers_with_mental_health_disorder
workplace_response
us_state_live
us_state_work


In [11]:
# Replace missing values in 'Total_employees' with '1-5'
df['Total_employees'] = df['Total_employees'].replace(to_replace=np.nan, value='1-5')

# Replace missing values in 'Tech_organization' with mode (mode returns a series, so we access the first value)
df['Tech_organization'] = df['Tech_organization'].fillna(df['Tech_organization'].mode()[0])

# Grouping by 'Self_employed' and filling missing values in each column with mode of the respective group
for column in df.columns:
    if column != 'Self_employed':
        # Use transform to ensure the result has the same index as the original DataFrame
        df[column] = df.groupby('Self_employed')[column].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))



In [12]:
df_no_drp = df

In [13]:
#Dropping remainig rows which still contained missing values
df=df.dropna()

In [14]:
df.isnull().sum().sum()

0

Encoding

In [15]:
import pandas as pd
import numpy as np

# Load the dataset (replace with the correct path if needed)
# df = pd.read_csv('/content/drive/MyDrive/OSMI/expanded_mental_health_10k.csv')

# Manually encode specific columns as per the provided mapping
No_of_pos = []
for i in df['work_pos']:
    No_of_pos.append(i.count('|') + 1)
df['No_of_pos'] = No_of_pos
df.drop(labels='work_pos', axis=1, inplace=True)

encode = {
    'Total_employees': [['26-100', '6-25', '0', 'More than 1000', '100-500', '500-1000', '1-5'], [3, 2, 0, 6, 4, 5, 1]],
    'Mental_health_coverage': [['Not eligible for coverage / N/A', 'No', 'Yes', "I don't know"], [0, 2, 3, 1]],
    'Awareness_of_options_under_coverage': [['Yes', 'No', 'I am not sure'], [2, 1, 0]],
    'Employers_discussion_on_mental_health': [['Yes', 'No', "I don't know"], [2, 1, 0]],
    'Resources_and_options_for_help': [['Yes', 'No', "I don't know"], [2, 1, 0]],
    'Anomity_of_employees_using_mental_health_resources': [['Yes', 'No', "I don't know"], [2, 1, 0]],
    'Mental_issue_related_leave': [['Somewhat easy', 'Neither easy nor difficult', 'Very easy', 'Very difficult', 'Somewhat difficult', "I don't know"], [4, 2, 5, 0, 1, 3]],
    'Discussing_mental_health_neg_consequences': [['No', 'Yes', 'Maybe'], [2, 0, 1]],
    'Discussing_physical_health_neg_consequences': [['No', 'Maybe', 'Yes'], [2, 1, 0]],
    'Comfort_discussing_with_coworkers': [['Maybe', 'Yes', 'No'], [1, 2, 0]],
    'Comfort_discussing_with_supervisor': [['Yes', 'No', 'Maybe'], [2, 0, 1]],
    'Employer_seriousness_on_mental_health': [['Yes', 'No', "I don't know"], [2, 0, 1]],
    'Negative_consequences_of_coworkers_with_mental_health_disorder': [['No', 'Yes'], [1, 0]],
    'prev_empl_mental_benefits': [['Yes, they all did', "I don't know", 'Some did', 'No, none did', 'Not Applicable'], [4, 2, 3, 0, 1]],
    'aware_benefits_prev_empl': [['I was aware of some', 'N/A (not currently aware)', 'Yes, I was aware of all of them', 'No, I only became aware later', 'Not Applicable'], [3, 2, 4, 0, 1]],
    'formal_discuss_mental_health_premp': [['None did', 'Some did', 'Not Applicable', "I don't know", 'Yes, they all did'], [0, 3, 1, 2, 4]],
    'premp_provide_resources_learn_seekhelp': [['Some did', 'None did', 'Not Applicable', 'Yes, they all did'], [2, 0, 1, 3]],
    'anonymity_protect_premp_choose_resources': [['Yes, always', "I don't know", 'Sometimes', 'No', 'Not Applicable'], [4, 2, 3, 0, 1]],
    'neg_eff_discuss_mental_health_premp': [['None of them', 'Some of them', 'Yes, all of them', "I don't know", 'Not Applicable'], [4, 1, 0, 2, 3]],
    'neg_eff_discuss_phy_health_premp': [['None of them', 'Some of them', 'Yes, all of them', 'Not Applicable'], [3, 1, 0, 2]],
    'willing_discuss_issue_prev_coworkers': [['No, at none of my previous employers', 'Some of my previous employers', 'Yes, at all of my previous employers', 'Not Applicable'], [0, 1, 3, 2]],
    'willing_discuss_issue_dir_supervisors': [['Some of my previous employers', 'Yes, at all of my previous employers', 'No, at none of my previous employers', "I don't know", 'Not Applicable'], [3, 4, 0, 1, 2]],
    'premp_took_menhealth_seriously_as_phyhealth': [["I don't know", 'Some did', 'None did', 'Yes, they all did', 'Not Applicable'], [0, 2, 1, 3, 0]],
    'neg_comments_abt_coworkers_with_menissues_prev_workplace': [['None of them', 'Some of them', 'Not Applicable', 'Yes, all of them'], [1, 2, 0, 3]],
    'bring_phyhealth_issue_with_potential_empl_interview': [['Maybe', 'Yes', 'No'], [1, 2, 0]],
    'bring_menhealth_issue_with_potential_empl_interview': [['Maybe', 'No', 'Yes'], [1, 0, 2]],
    'hurt_carrer_if_identified_as_person_with_mental_issue': [['Maybe', "No, I don't think it would", 'Yes, I think it would', 'No, it has not', 'Yes, it has'], [1, 0, 2, 0, 2]],
    'more_neg_view_from_coworker_if_know_abt_men_issue': [["No, I don't think they would", 'Maybe', 'Yes, they do', 'Yes, I think they would', 'No, they do not'], [0, 1, 2, 2, 0]],
    'share_w_friends_fam': [['Somewhat open', 'Neutral', 'Not applicable to me (I do not have a mental illness)', 'Very open', 'Not open at all', 'Somewhat not open'], [4, 3, 0, 5, 1, 2]],
    'workplace_response': [['No', 'Maybe/Not sure', 'Yes, I experienced', 'Yes, I observed', np.nan], [0, 1, 2, 2, 1]],
    'fam_hist': [['No', 'Yes', "I don't know"], [0, 2, 1]],
    'past_mh': [['Yes', 'Maybe', 'No'], [2, 1, 0]],
    'curr_mh': [['No', 'Yes', 'Maybe'], [0, 2, 1]],
    'diagnosed_prof': [['Yes', 'No'], [1, 0]],
    'work_interference_treatment': [['Not applicable to me', 'Rarely', 'Sometimes', 'Never', 'Often'], [0, 2, 3, 1, 4]],
    'work_interference_no_treatment': [['Not applicable to me', 'Sometimes', 'Often', 'Rarely', 'Never'], [0, 3, 4, 2, 1]],
    'remote': [['Sometimes', 'Never', 'Always'], [1, 0, 2]]
}

# Apply the manual encoding
for col, mapping in encode.items():
    df[col] = df[col].replace(to_replace=mapping[0], value=mapping[1])

# Dividing states of the USA into 4 divisions
north_east = ['Pennsylvania', 'New York', 'Rhode Island', 'Maine', 'New Jersey', 'New Hampshire', 'Massachusetts', 'Vermont', 'Connecticut']
south = ['Delaware', 'District of Columbia', 'Texas', 'Louisiana', 'Oklahoma', 'Alabama', 'Kentucky', 'Virginia', 'South Carolina', 'Maryland', 'West Virginia', 'North Carolina', 'Georgia', 'Florida', 'Tennessee']
mid_west = ['Illinois', 'Indiana', 'Minnesota', 'Iowa', 'Ohio', 'Michigan', 'Wisconsin', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota', 'Kansas']
west = ['California', 'Montana', 'Oregon', 'New Mexico', 'Alaska', 'Washington', 'Nevada', 'Arizona', 'Utah', 'Colorado', 'Hawaii', 'Idaho']

# Encoding columns for us_state_live and us_state_work
# north_east=1, mid_west=2, west=3, south=4
df['us_state_live'].replace(to_replace=north_east, value=1, inplace=True)
df['us_state_live'].replace(to_replace=mid_west, value=2, inplace=True)
df['us_state_live'].replace(to_replace=west, value=3, inplace=True)
df['us_state_live'].replace(to_replace=south, value=4, inplace=True)

df['us_state_work'].replace(to_replace=north_east, value=1, inplace=True)
df['us_state_work'].replace(to_replace=mid_west, value=2, inplace=True)
df['us_state_work'].replace(to_replace=west, value=3, inplace=True)
df['us_state_work'].replace(to_replace=south, value=4, inplace=True)

# Encoding for country_live and country_work manually (USA = 0, UK = 1)
df['country_live'] = df['country_live'].replace({'USA': 0, 'UK': 1})
df['country_work'] = df['country_work'].replace({'USA': 0, 'UK': 1})

# Automatic encoding of remaining string values
# This function will handle any remaining string values and map them to unique integers
def encode_remaining_strings(df, columns):
    for col in columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('category').cat.codes
    return df

# Identify columns that still have string values
string_columns = df.select_dtypes(include=['object']).columns

# Apply the automatic encoding to remaining string columns
df = encode_remaining_strings(df, string_columns)

# Saving the encoded DataFrame to a new CSV file
df.to_csv('prepro_clean_mental_health_encoded_complete.csv', index=False)

print("Encoding completed and file saved as 'prepro_clean_mental_health_encoded_complete.csv'.")


Encoding completed and file saved as 'prepro_clean_mental_health_encoded_complete.csv'.


In [16]:
#Unique values in each column
for i in df.columns:
    print(i," : ",df[i].unique())
print(len(df.columns))

Self_employed  :  [0]
Total_employees  :  [3 2 6 4 5 1]
Tech_organization  :  [1. 0.]
Mental_health_coverage  :  [0 2 3 1]
Awareness_of_options_under_coverage  :  [1 2 0]
Employers_discussion_on_mental_health  :  [1 2 0]
Resources_and_options_for_help  :  [1 2 0]
Anomity_of_employees_using_mental_health_resources  :  [0 2 1]
Mental_issue_related_leave  :  [5 4 2 0 1 3]
Discussing_mental_health_neg_consequences  :  [2 1 0]
Discussing_physical_health_neg_consequences  :  [2 1 0]
Comfort_discussing_with_coworkers  :  [1 2 0]
Comfort_discussing_with_supervisor  :  [2 1 0]
Employer_seriousness_on_mental_health  :  [1 2 0]
Negative_consequences_of_coworkers_with_mental_health_disorder  :  [1 0]
have_prev_empl  :  [1 0]
prev_empl_mental_benefits  :  [0 4 2 3 1]
aware_benefits_prev_empl  :  [2 3 4 0 1]
formal_discuss_mental_health_premp  :  [2 0 3 1 4]
premp_provide_resources_learn_seekhelp  :  [0 2 1 3]
anonymity_protect_premp_choose_resources  :  [2 4 3 0 1]
neg_eff_discuss_mental_health_pre